#### Daftar Isi:
- [5.1 Pemodelan Tahap 1](#51-pemodelan-svm-tahap-1)
- [6.1 Pengukuran Performa Tahap 1](#61-pengukuran-performa-tahap-1)
    - [6.1.1 Pengukuran Performa Tahap 1: SVM + BOW](#611-pengukuran-performa-tahap-1-svm--bow)
    - [6.1.2 Pengukuran Performa Tahap 1: SVM + TF-IDF](#612-pengukuran-performa-tahap-1-svm--tf-idf)
- [5.2 Pemodelan Tahap 2](#52-pemodelan-svm-tahap-2)
- [6.2 Pengukuran Performa Tahap 2](#62-pengukuran-performa-tahap-2)

***

In [1]:
from IPython.core.interactiveshell import InteractiveShell
import sklearnex

InteractiveShell.ast_node_interactivity = "all"
sklearnex.patch_sklearn()

%load_ext watermark
%watermark -a "F. Waskito" -n -t -u -v

Author: F. Waskito

Last updated: Thu Jun 08 2023 13:05:09

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 8.12.0



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas
from collection import analysis

path = "data/tweet/depresi_or_bipolar_tweets_id_01-10.csv"
tweets_table = pandas.read_csv(path)
texts = tweets_table.loc[:, "Text"].copy().to_list()
labels = tweets_table.loc[:, "Sentiment"].copy().to_list()

analysis.get_shape(texts)
analysis.get_distribution(labels)

Shape: (3409,)
Distribution:
	('positive', 864)
	('neutral', 1253)
	('negative', 1292)


Prapemrosesan

In [3]:
import time
from tqdm import tqdm
from preprocess.preprocessing import TextPreprocessor

preprocessor = TextPreprocessor()
for i, text in enumerate(tqdm(texts)):
    text = preprocessor.clean(text)
    text = preprocessor.standardize(text)
    tokens = preprocessor.tokenize(text)
    tokens = preprocessor.filter(tokens)
    texts[i] = preprocessor.stem(tokens)
    time.sleep(0.001)

100%|██████████| 3409/3409 [11:09<00:00,  5.09it/s]


Ekstraksi Fitur

In [4]:
from preprocess.feature.extraction import TextVectorizer

In [5]:
# BOW
extractor = TextVectorizer(texts)
extractor.transform(target="bow", min_df=2, norm=True)
vector_texts = extractor.vectors

analysis.get_shape(vector_texts)

Shape: (3409, 2205)


In [5]:
# TF-IDF
extractor = TextVectorizer(texts)
extractor.transform(target="tfidf", min_df=2, norm=True)
vector_texts = extractor.vectors

analysis.get_shape(vector_texts)

Shape: (3409, 2205)


Transformasi Label

In [6]:
from preprocess.encoding import LabelEncoder

In [7]:
encoder = LabelEncoder(labels)
encoder.transform(target="integer")
encoded_labels = encoder.encoded_labels

analysis.get_distribution(encoded_labels)

Distribution:
	(2, 864)
	(1, 1253)
	(0, 1292)


Seprasi Data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    vector_texts,
    encoded_labels,
    test_size = 0.3,
    random_state = 42
)

print("> Train set:")
analysis.get_shape(X_train)
analysis.get_distribution(y_train)
print("\n> Test set:")
analysis.get_shape(X_test)
analysis.get_distribution(y_test)

> Train set:
Shape: (2386, 2205)
Distribution:
	(1, 866)
	(2, 619)
	(0, 901)

> Test set:
Shape: (1023, 2205)
Distribution:
	(0, 391)
	(1, 387)
	(2, 245)


***

### 5.1 Pemodelan SVM Tahap 1

In [10]:
from sklearn.svm import SVC

In [26]:
c = 10.0
gamma = 1.0
degree = 9

In [321]:
linear_svm = SVC(kernel="linear", C=c)

In [386]:
rbf_svm = SVC(kernel="rbf",C=c, gamma=gamma)

In [27]:
poly_svm = SVC(kernel="poly", C=c, gamma=gamma, degree=degree)

### 6.1 Pengukuran Performa Tahap 1

Beralih ke:
- [5.1 Pemodelan SVM Tahap 1](#51-pemodelan-svm-tahap-1)
- [6.1.1 Pengukuran Performa Tahap 1: SVM + BOW](#611-pengukuran-performa-tahap-1-svm--bow)
- [6.1.2 Pengukuran Performa Tahap 1: SVM + TF-IDF](#612-pengukuran-performa-tahap-1-svm--tf-idf)

In [14]:
from validation.cross import ImbalancedCV

In [28]:
n_fold = 5
scoring = ["accuracy", "precision", "recall", "f1",]
random_state = 42

In [324]:
linear_perform = ImbalancedCV(
    model = linear_svm,
    n_fold = n_fold,
    scoring = scoring,
    scoring_avg = "macro",
    random_state = random_state,
)

linear_perform.validate(X_train, y_train)

CV: 100%|██████████| 5/5 [00:16<00:00,  3.36s/it]


In [388]:
rbf_perform = ImbalancedCV(
    model = rbf_svm,
    n_fold = n_fold,
    scoring = scoring,
    scoring_avg = "macro",
    random_state = random_state,
)

rbf_perform.validate(X_train, y_train)

CV: 100%|██████████| 5/5 [00:17<00:00,  3.51s/it]


In [29]:
poly_perform = ImbalancedCV(
    model = poly_svm,
    n_fold = n_fold,
    scoring = scoring,
    scoring_avg = "macro",
    random_state = random_state,
)

poly_perform.validate(X_train, y_train)

CV: 100%|██████████| 5/5 [00:16<00:00,  3.32s/it]


#### __6.1.1 Pengukuran Performa Tahap 1: SVM + BOW__

Beralih ke:
- [6.1.1.2 Performa SVM-RBF](#6112-performa-svm-rbf)
- [6.1.1.3 Performa SVM-Polinomial](#6113-performa-svm-polinomial)
- [6.1.2 Pengukuran Performa Tahap 1: SVM + TF-IDF](#612-pengukuran-performa-tahap-1-svm--tf-idf)

##### 6.1.1.1 Performa SVM-Linier

In [22]:
linear_perform.get_score() # C= 1.0

{'mean_accuracy': 0.672,
 'mean_precision': 0.662,
 'mean_recall': 0.657,
 'mean_f1': 0.654}

In [33]:
linear_perform.get_score() # C= 10.0

{'mean_accuracy': 0.679,
 'mean_precision': 0.667,
 'mean_recall': 0.665,
 'mean_f1': 0.665}

In [43]:
linear_perform.get_score() # C= 100.0

{'mean_accuracy': 0.653,
 'mean_precision': 0.641,
 'mean_recall': 0.639,
 'mean_f1': 0.639}

In [53]:
linear_perform.get_score() # C= 1000.0

{'mean_accuracy': 0.642,
 'mean_precision': 0.632,
 'mean_recall': 0.629,
 'mean_f1': 0.63}

In [64]:
linear_perform.get_score() # C= 10000.0

{'mean_accuracy': 0.636,
 'mean_precision': 0.627,
 'mean_recall': 0.623,
 'mean_f1': 0.623}

##### 6.1.1.2 Performa SVM-RBF

In [23]:
rbf_perform.get_score() # C= 1.0; gamma= 0.01

{'mean_accuracy': 0.531,
 'mean_precision': 0.596,
 'mean_recall': 0.511,
 'mean_f1': 0.476}

In [34]:
rbf_perform.get_score() # C= 10.0; gamma= 0.01

{'mean_accuracy': 0.666,
 'mean_precision': 0.673,
 'mean_recall': 0.649,
 'mean_f1': 0.645}

In [44]:
rbf_perform.get_score() # C= 100.0; gamma= 0.01

{'mean_accuracy': 0.676,
 'mean_precision': 0.664,
 'mean_recall': 0.66,
 'mean_f1': 0.659}

In [54]:
rbf_perform.get_score() # C= 1000.0; gamma= 0.01

{'mean_accuracy': 0.679,
 'mean_precision': 0.666,
 'mean_recall': 0.66,
 'mean_f1': 0.661}

In [65]:
rbf_perform.get_score() # C= 10,000.0; gamma= 0.01

{'mean_accuracy': 0.647,
 'mean_precision': 0.636,
 'mean_recall': 0.631,
 'mean_f1': 0.632}

In [74]:
rbf_perform.get_score() # C= 1.0; gamma= 0.1

{'mean_accuracy': 0.648,
 'mean_precision': 0.651,
 'mean_recall': 0.627,
 'mean_f1': 0.625}

In [81]:
rbf_perform.get_score() # C= 10.0; gamma= 0.1

{'mean_accuracy': 0.68,
 'mean_precision': 0.669,
 'mean_recall': 0.661,
 'mean_f1': 0.661}

In [88]:
rbf_perform.get_score() # C= 100.0; gamma= 0.1

{'mean_accuracy': 0.678,
 'mean_precision': 0.665,
 'mean_recall': 0.658,
 'mean_f1': 0.658}

In [95]:
rbf_perform.get_score() # C= 1000.0; gamma= 0.1

{'mean_accuracy': 0.647,
 'mean_precision': 0.636,
 'mean_recall': 0.627,
 'mean_f1': 0.628}

In [102]:
rbf_perform.get_score() # C= 10,000.0; gamma= 0.1

{'mean_accuracy': 0.625,
 'mean_precision': 0.613,
 'mean_recall': 0.603,
 'mean_f1': 0.603}

In [109]:
rbf_perform.get_score() # C= 1.0; gamma= 1.0

{'mean_accuracy': 0.575,
 'mean_precision': 0.612,
 'mean_recall': 0.531,
 'mean_f1': 0.509}

In [116]:
rbf_perform.get_score() # C= 10.0; gamma= 1.0

{'mean_accuracy': 0.581,
 'mean_precision': 0.608,
 'mean_recall': 0.538,
 'mean_f1': 0.518}

In [123]:
rbf_perform.get_score() # C= 100.0; gamma= 1.0

{'mean_accuracy': 0.572,
 'mean_precision': 0.597,
 'mean_recall': 0.53,
 'mean_f1': 0.513}

In [130]:
rbf_perform.get_score() # C= 1000.0; gamma= 1.0

{'mean_accuracy': 0.57,
 'mean_precision': 0.601,
 'mean_recall': 0.528,
 'mean_f1': 0.511}

In [137]:
rbf_perform.get_score() # C= 10,000.0; gamma= 1.0

{'mean_accuracy': 0.57,
 'mean_precision': 0.601,
 'mean_recall': 0.528,
 'mean_f1': 0.51}

##### 6.1.1.3 Performa SVM-Polinomial

In [24]:
poly_perform.get_score() # C= 1.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.365,
 'mean_precision': 0.305,
 'mean_recall': 0.336,
 'mean_f1': 0.182}

In [35]:
poly_perform.get_score() # C= 10.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.365,
 'mean_precision': 0.305,
 'mean_recall': 0.336,
 'mean_f1': 0.182}

In [45]:
poly_perform.get_score() # C= 100.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.365,
 'mean_precision': 0.305,
 'mean_recall': 0.336,
 'mean_f1': 0.182}

In [55]:
poly_perform.get_score() # C= 1000.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.368,
 'mean_precision': 0.566,
 'mean_recall': 0.338,
 'mean_f1': 0.188}

In [66]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.383,
 'mean_precision': 0.785,
 'mean_recall': 0.357,
 'mean_f1': 0.225}

In [75]:
poly_perform.get_score() # C= 1.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.368,
 'mean_precision': 0.566,
 'mean_recall': 0.338,
 'mean_f1': 0.188}

In [82]:
poly_perform.get_score() # C= 10.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.383,
 'mean_precision': 0.785,
 'mean_recall': 0.357,
 'mean_f1': 0.225}

In [89]:
poly_perform.get_score() # C= 100.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.405,
 'mean_precision': 0.706,
 'mean_recall': 0.379,
 'mean_f1': 0.269}

In [96]:
poly_perform.get_score() # C= 1000.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.495,
 'mean_precision': 0.606,
 'mean_recall': 0.473,
 'mean_f1': 0.432}

In [103]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.565,
 'mean_precision': 0.602,
 'mean_recall': 0.546,
 'mean_f1': 0.53}

In [110]:
poly_perform.get_score() # C= 1.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.495,
 'mean_precision': 0.606,
 'mean_recall': 0.473,
 'mean_f1': 0.432}

In [117]:
poly_perform.get_score() # C= 10.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.565,
 'mean_precision': 0.602,
 'mean_recall': 0.546,
 'mean_f1': 0.53}

In [124]:
poly_perform.get_score() # C= 100.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.599,
 'mean_precision': 0.597,
 'mean_recall': 0.579,
 'mean_f1': 0.57}

In [131]:
poly_perform.get_score() # C= 1000.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.595,
 'mean_precision': 0.582,
 'mean_recall': 0.573,
 'mean_f1': 0.567}

In [138]:
poly_perform.get_score() # C= 10,000.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.577,
 'mean_precision': 0.559,
 'mean_recall': 0.558,
 'mean_f1': 0.554}

In [142]:
poly_perform.get_score() # C= 1.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [146]:
poly_perform.get_score() # C= 10.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [150]:
poly_perform.get_score() # C= 100.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [154]:
poly_perform.get_score() # C= 1000.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [158]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [162]:
poly_perform.get_score() # C= 1.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.364,
 'mean_precision': 0.321,
 'mean_recall': 0.334,
 'mean_f1': 0.18}

In [166]:
poly_perform.get_score() # C= 10.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.368,
 'mean_precision': 0.566,
 'mean_recall': 0.338,
 'mean_f1': 0.188}

In [170]:
poly_perform.get_score() # C= 100.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.373,
 'mean_precision': 0.709,
 'mean_recall': 0.344,
 'mean_f1': 0.2}

In [174]:
poly_perform.get_score() # C= 1000.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.38,
 'mean_precision': 0.782,
 'mean_recall': 0.353,
 'mean_f1': 0.218}

In [178]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.385,
 'mean_precision': 0.785,
 'mean_recall': 0.359,
 'mean_f1': 0.229}

In [182]:
poly_perform.get_score() # C= 1.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.412,
 'mean_precision': 0.619,
 'mean_recall': 0.388,
 'mean_f1': 0.29}

In [186]:
poly_perform.get_score() # C= 10.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.429,
 'mean_precision': 0.596,
 'mean_recall': 0.404,
 'mean_f1': 0.322}

In [190]:
poly_perform.get_score() # C= 100.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.463,
 'mean_precision': 0.574,
 'mean_recall': 0.443,
 'mean_f1': 0.387}

In [194]:
poly_perform.get_score() # C= 1000.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.524,
 'mean_precision': 0.59,
 'mean_recall': 0.504,
 'mean_f1': 0.477}

In [198]:
poly_perform.get_score() # C= 10,000.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.546,
 'mean_precision': 0.582,
 'mean_recall': 0.528,
 'mean_f1': 0.509}

In [209]:
poly_perform.get_score() # C= 1.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [213]:
poly_perform.get_score() # C= 10.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [217]:
poly_perform.get_score() # C= 100.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [221]:
poly_perform.get_score() # C= 1000.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [225]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.363,
 'mean_precision': 0.188,
 'mean_recall': 0.334,
 'mean_f1': 0.178}

In [229]:
poly_perform.get_score() # C= 1.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.364,
 'mean_precision': 0.321,
 'mean_recall': 0.334,
 'mean_f1': 0.18}

In [233]:
poly_perform.get_score() # C= 10.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.364,
 'mean_precision': 0.321,
 'mean_recall': 0.334,
 'mean_f1': 0.18}

In [237]:
poly_perform.get_score() # C= 100.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.368,
 'mean_precision': 0.633,
 'mean_recall': 0.339,
 'mean_f1': 0.19}

In [241]:
poly_perform.get_score() # C= 1000.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.369,
 'mean_precision': 0.638,
 'mean_recall': 0.34,
 'mean_f1': 0.192}

In [245]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.38,
 'mean_precision': 0.782,
 'mean_recall': 0.352,
 'mean_f1': 0.217}

In [249]:
poly_perform.get_score() # C= 1.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.404,
 'mean_precision': 0.622,
 'mean_recall': 0.379,
 'mean_f1': 0.274}

In [253]:
poly_perform.get_score() # C= 10.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.406,
 'mean_precision': 0.578,
 'mean_recall': 0.382,
 'mean_f1': 0.281}

In [257]:
poly_perform.get_score() # C= 100.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.42,
 'mean_precision': 0.587,
 'mean_recall': 0.396,
 'mean_f1': 0.307}

In [261]:
poly_perform.get_score() # C= 1000.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.441,
 'mean_precision': 0.564,
 'mean_recall': 0.42,
 'mean_f1': 0.35}

In [265]:
poly_perform.get_score() # C= 10,000.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.47,
 'mean_precision': 0.561,
 'mean_recall': 0.448,
 'mean_f1': 0.399}

Beralih ke:
- [5.1 Pemodealn SVM Tahap 1](#51-pemodelan-svm-tahap-1)

#### __6.1.2 Pengukuran Performa Tahap 1: SVM + TF-IDF__

Beralih ke:
- [6.1.2.2 Performa SVM RBF](#6122-performa-svm-rbf)
- [6.1.2.3 Performa SVM-Polinomial](#6123-performa-svm-polinomial)

##### 6.1.2.1 Performa SVM-Linier

In [277]:
linear_perform.get_score() # C= 1.0

{'mean_accuracy': 0.696,
 'mean_precision': 0.685,
 'mean_recall': 0.682,
 'mean_f1': 0.683}

In [287]:
linear_perform.get_score() # C= 10.0

{'mean_accuracy': 0.676,
 'mean_precision': 0.666,
 'mean_recall': 0.662,
 'mean_f1': 0.663}

In [297]:
linear_perform.get_score() # C= 100.0

{'mean_accuracy': 0.659,
 'mean_precision': 0.651,
 'mean_recall': 0.643,
 'mean_f1': 0.645}

In [307]:
linear_perform.get_score() # C= 1000.0

{'mean_accuracy': 0.653,
 'mean_precision': 0.641,
 'mean_recall': 0.636,
 'mean_f1': 0.636}

In [317]:
linear_perform.get_score() # C= 10,000.0

{'mean_accuracy': 0.632,
 'mean_precision': 0.627,
 'mean_recall': 0.618,
 'mean_f1': 0.618}

##### 6.1.2.2 Performa SVM-RBF

In [278]:
rbf_perform.get_score() # C= 1.0; gamma= 0.01

{'mean_accuracy': 0.467,
 'mean_precision': 0.657,
 'mean_recall': 0.476,
 'mean_f1': 0.408}

In [288]:
rbf_perform.get_score() # C= 10.0; gamma= 0.01

{'mean_accuracy': 0.68,
 'mean_precision': 0.676,
 'mean_recall': 0.674,
 'mean_f1': 0.673}

In [298]:
rbf_perform.get_score() # C= 100.0; gamma= 0.01

{'mean_accuracy': 0.698,
 'mean_precision': 0.687,
 'mean_recall': 0.683,
 'mean_f1': 0.684}

In [308]:
rbf_perform.get_score() # C= 1000.0; gamma= 0.01

{'mean_accuracy': 0.668,
 'mean_precision': 0.66,
 'mean_recall': 0.654,
 'mean_f1': 0.655}

In [318]:
rbf_perform.get_score() # C= 10,000.0; gamma= 0.01

{'mean_accuracy': 0.658,
 'mean_precision': 0.648,
 'mean_recall': 0.642,
 'mean_f1': 0.643}

In [327]:
rbf_perform.get_score() # C= 1.0; gamma= 0.1

{'mean_accuracy': 0.675,
 'mean_precision': 0.678,
 'mean_recall': 0.668,
 'mean_f1': 0.668}

In [334]:
rbf_perform.get_score() # C= 10.0; gamma= 0.1

{'mean_accuracy': 0.701,
 'mean_precision': 0.691,
 'mean_recall': 0.686,
 'mean_f1': 0.687}

In [341]:
rbf_perform.get_score() # C= 100.0; gamma= 0.1

{'mean_accuracy': 0.684,
 'mean_precision': 0.675,
 'mean_recall': 0.67,
 'mean_f1': 0.671}

In [348]:
rbf_perform.get_score() # C= 1000.0; gamma= 0.1

{'mean_accuracy': 0.671,
 'mean_precision': 0.662,
 'mean_recall': 0.657,
 'mean_f1': 0.658}

In [355]:
rbf_perform.get_score() # C= 10,000.0; gamma= 0.1

{'mean_accuracy': 0.67,
 'mean_precision': 0.661,
 'mean_recall': 0.655,
 'mean_f1': 0.656}

In [362]:
rbf_perform.get_score() # C= 1.0; gamma= 1.0

{'mean_accuracy': 0.701,
 'mean_precision': 0.708,
 'mean_recall': 0.674,
 'mean_f1': 0.676}

In [369]:
rbf_perform.get_score() # C= 10.0; gamma= 1.0

{'mean_accuracy': 0.714,
 'mean_precision': 0.711,
 'mean_recall': 0.692,
 'mean_f1': 0.695}

In [376]:
rbf_perform.get_score() # C= 100.0; gamma= 1.0

{'mean_accuracy': 0.713,
 'mean_precision': 0.711,
 'mean_recall': 0.691,
 'mean_f1': 0.694}

In [383]:
rbf_perform.get_score() # C= 1000.0; gamma= 1.0

{'mean_accuracy': 0.711,
 'mean_precision': 0.709,
 'mean_recall': 0.69,
 'mean_f1': 0.692}

In [390]:
rbf_perform.get_score() # C= 10,000.0; gamma= 1.0

{'mean_accuracy': 0.711,
 'mean_precision': 0.709,
 'mean_recall': 0.69,
 'mean_f1': 0.692}

##### 6.1.2.3 Performa SVM-Polinomial

In [279]:
poly_perform.get_score() # C= 1.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.43,
 'mean_precision': 0.766,
 'mean_recall': 0.387,
 'mean_f1': 0.288}

In [289]:
poly_perform.get_score() # C= 10.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.43,
 'mean_precision': 0.766,
 'mean_recall': 0.387,
 'mean_f1': 0.288}

In [299]:
poly_perform.get_score() # C= 100.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.43,
 'mean_precision': 0.766,
 'mean_recall': 0.387,
 'mean_f1': 0.288}

In [309]:
poly_perform.get_score() # C= 1000.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.43,
 'mean_precision': 0.766,
 'mean_recall': 0.387,
 'mean_f1': 0.288}

In [319]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.01; degree= 3

{'mean_accuracy': 0.43,
 'mean_precision': 0.766,
 'mean_recall': 0.387,
 'mean_f1': 0.288}

In [328]:
poly_perform.get_score() # C= 1.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.43,
 'mean_precision': 0.766,
 'mean_recall': 0.387,
 'mean_f1': 0.288}

In [335]:
poly_perform.get_score() # C= 10.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.43,
 'mean_precision': 0.766,
 'mean_recall': 0.387,
 'mean_f1': 0.288}

In [342]:
poly_perform.get_score() # C= 100.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.44,
 'mean_precision': 0.769,
 'mean_recall': 0.395,
 'mean_f1': 0.302}

In [349]:
poly_perform.get_score() # C= 1000.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.567,
 'mean_precision': 0.652,
 'mean_recall': 0.528,
 'mean_f1': 0.511}

In [356]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.1; degree= 3

{'mean_accuracy': 0.575,
 'mean_precision': 0.625,
 'mean_recall': 0.592,
 'mean_f1': 0.578}

In [363]:
poly_perform.get_score() # C= 1.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.567,
 'mean_precision': 0.652,
 'mean_recall': 0.528,
 'mean_f1': 0.511}

In [370]:
poly_perform.get_score() # C= 10.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.575,
 'mean_precision': 0.625,
 'mean_recall': 0.592,
 'mean_f1': 0.578}

In [377]:
poly_perform.get_score() # C= 100.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.576,
 'mean_precision': 0.625,
 'mean_recall': 0.593,
 'mean_f1': 0.578}

In [384]:
poly_perform.get_score() # C= 1000.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.575,
 'mean_precision': 0.626,
 'mean_recall': 0.592,
 'mean_f1': 0.577}

In [391]:
poly_perform.get_score() # C= 10,000.0; gamma= 1.0; degree= 3

{'mean_accuracy': 0.575,
 'mean_precision': 0.626,
 'mean_recall': 0.592,
 'mean_f1': 0.577}

In [395]:
poly_perform.get_score() # C= 1.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [399]:
poly_perform.get_score() # C= 10.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [403]:
poly_perform.get_score() # C= 100.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [407]:
poly_perform.get_score() # C= 1000.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [411]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.01; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [415]:
poly_perform.get_score() # C= 1.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [419]:
poly_perform.get_score() # C= 10.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [423]:
poly_perform.get_score() # C= 100.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [427]:
poly_perform.get_score() # C= 1000.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [431]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.1; degree= 6

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.384,
 'mean_f1': 0.281}

In [435]:
poly_perform.get_score() # C= 1.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.491,
 'mean_precision': 0.7,
 'mean_recall': 0.445,
 'mean_f1': 0.386}

In [439]:
poly_perform.get_score() # C= 10.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.358,
 'mean_precision': 0.69,
 'mean_recall': 0.422,
 'mean_f1': 0.301}

In [443]:
poly_perform.get_score() # C= 100.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.358,
 'mean_precision': 0.69,
 'mean_recall': 0.422,
 'mean_f1': 0.301}

In [447]:
poly_perform.get_score() # C= 1000.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.358,
 'mean_precision': 0.69,
 'mean_recall': 0.422,
 'mean_f1': 0.301}

In [451]:
poly_perform.get_score() # C= 10,000.0; gamma= 1.0; degree= 6

{'mean_accuracy': 0.358,
 'mean_precision': 0.69,
 'mean_recall': 0.422,
 'mean_f1': 0.301}

In [456]:
poly_perform.get_score() # C= 1.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.55,
 'mean_precision': 0.559,
 'mean_recall': 0.535,
 'mean_f1': 0.531}

In [460]:
poly_perform.get_score() # C= 10.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.55,
 'mean_precision': 0.559,
 'mean_recall': 0.535,
 'mean_f1': 0.531}

In [465]:
poly_perform.get_score() # C= 100.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.428,
 'mean_precision': 0.773,
 'mean_recall': 0.383,
 'mean_f1': 0.28}

In [469]:
poly_perform.get_score() # C= 1000.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.429,
 'mean_precision': 0.774,
 'mean_recall': 0.385,
 'mean_f1': 0.283}

In [473]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.01; degree= 9

{'mean_accuracy': 0.429,
 'mean_precision': 0.774,
 'mean_recall': 0.385,
 'mean_f1': 0.283}

In [477]:
poly_perform.get_score() # C= 1.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.429,
 'mean_precision': 0.774,
 'mean_recall': 0.385,
 'mean_f1': 0.283}

In [481]:
poly_perform.get_score() # C= 10.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.429,
 'mean_precision': 0.774,
 'mean_recall': 0.385,
 'mean_f1': 0.283}

In [485]:
poly_perform.get_score() # C= 100.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.429,
 'mean_precision': 0.774,
 'mean_recall': 0.385,
 'mean_f1': 0.283}

In [489]:
poly_perform.get_score() # C= 1000.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.429,
 'mean_precision': 0.774,
 'mean_recall': 0.385,
 'mean_f1': 0.283}

In [493]:
poly_perform.get_score() # C= 10,000.0; gamma= 0.1; degree= 9

{'mean_accuracy': 0.429,
 'mean_precision': 0.774,
 'mean_recall': 0.385,
 'mean_f1': 0.283}

In [497]:
poly_perform.get_score() # C= 1.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.478,
 'mean_precision': 0.723,
 'mean_recall': 0.432,
 'mean_f1': 0.365}

In [501]:
poly_perform.get_score() # C= 10.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.348,
 'mean_precision': 0.721,
 'mean_recall': 0.414,
 'mean_f1': 0.285}

In [17]:
poly_perform.get_score() # C= 100.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.349,
 'mean_precision': 0.719,
 'mean_recall': 0.414,
 'mean_f1': 0.285}

In [21]:
poly_perform.get_score() # C= 1000.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.349,
 'mean_precision': 0.719,
 'mean_recall': 0.414,
 'mean_f1': 0.285}

In [25]:
poly_perform.get_score() # C= 10,000.0; gamma= 1.0; degree= 9

{'mean_accuracy': 0.349,
 'mean_precision': 0.719,
 'mean_recall': 0.414,
 'mean_f1': 0.285}

Brealih ke:
- [5.1 Pemodelan SVM Tahap 1](#51-pemodelan-svm-tahap-1)

***

### 5.2 Pemodelan SVM Tahap 2

### 6.2 Pengukuran Performa Tahap 2

#### __6.2.1 Pengukuran Performa SVM + BOW__

#### __6.2.2 Pengukuran Performa SVM + TF-IDF__